# 阶段一: 候选短语的自动化发现 (The Discovery Engine)

**总目标:** 从约17万条中国新闻中，通过三种互补的算法（Gensim Phrases, spaCy NER, spaCy Noun Chunks），大规模挖掘潜在的、有意义的多词短语，为后续的人工审核环节提供一个高质量的“候选池”。

---
### 步骤 1.0 - 环境设置与库导入

In [1]:
# --- 步骤 1.0: 环境设置与库导入 ---

# 导入必要的库
import pandas as pd
import spacy
import time
import re
import os
from gensim.models.phrases import Phrases
from collections import Counter
from tqdm.auto import tqdm
import pickle
import multiprocessing as mp
import psutil

# --- 快速测试模式开关 ---
TEST_MODE = True
TEST_SAMPLE_SIZE = 1000

# --- 配置区 ---
ALIYUN_OSS_PATH = ''
# 输入文件路径
INPUT_CHINA_DATA_PATH = os.path.join(ALIYUN_OSS_PATH, "../data_processed/final_china_news.csv")

# 中间产出文件路径
TOKEN_LISTS_PATH = os.path.join(ALIYUN_OSS_PATH, "../data_processed/intermediate_raw_token_lists.pkl")
CANDIDATES_GENSIM_PATH = os.path.join(ALIYUN_OSS_PATH, "../data_processed/candidates_gensim.pkl")
CANDIDATES_NER_PATH = os.path.join(ALIYUN_OSS_PATH, "../data_processed/candidates_ner.pkl")
# CANDIDATES_NOUN_CHUNKS_PATH 已被移除

# --- 并行处理配置 ---
cpu_cores = psutil.cpu_count(logical=False)
N_PROCESSES = min(cpu_cores - 1 if cpu_cores > 1 else 1, 8)
if N_PROCESSES < 1: N_PROCESSES = 1

print("--- 环境准备 ---")
if TEST_MODE:
    print(f"🚀🚀🚀 运行在【快速测试模式】下，将处理前 {TEST_SAMPLE_SIZE} 行数据！🚀🚀🚀")
else:
    print("🚢🚢🚢 运行在【完整数据模式】下，将处理所有数据。🚢🚢🚢")
print(f"输入文件路径: {INPUT_CHINA_DATA_PATH}")
print(f"将使用 {N_PROCESSES} 个进程进行并行处理。")

# --- 加载spaCy模型 ---
print("\n正在加载spaCy模型 'en_core_web_lg' (完整模式)...")
start_time = time.time()
try:
    nlp = spacy.load("en_core_web_lg")
    print(f"spaCy模型加载成功！耗时: {time.time() - start_time:.2f} 秒")
except OSError:
    print("错误: spaCy模型 'en_core_web_lg' 未安装。")
    print("请在你的终端或命令行中运行: python -m spacy download en_core_web_lg")
    nlp = None

--- 环境准备 ---
🚀🚀🚀 运行在【快速测试模式】下，将处理前 1000 行数据！🚀🚀🚀
输入文件路径: ../data_processed/final_china_news.csv
将使用 5 个进程进行并行处理。

正在加载spaCy模型 'en_core_web_lg' (完整模式)...
spaCy模型加载成功！耗时: 1.16 秒


### 步骤 1.1 - 文本预分词

需求: 为 Gensim 准备 raw_token_lists。
实现: 使用并行化的 nlp.pipe，但只启用分词功能以获得最高速度。

In [2]:
# --- 步骤 1.1: 并行预分词 ---

# 1. 加载数据
print("--- 步骤 1.1a: 数据加载 ---")
start_time_load = time.time()
df = pd.DataFrame()
try:
    read_nrows = TEST_SAMPLE_SIZE if TEST_MODE else None
    df = pd.read_csv(INPUT_CHINA_DATA_PATH, dtype=str, nrows=read_nrows)
    df.dropna(subset=['CONTENT'], inplace=True)
    print(f"数据加载完成！耗时: {time.time() - start_time_load:.2f} 秒。 数据形状: {df.shape}")
except FileNotFoundError:
    print(f"❌ 错误: 输入文件不存在: {INPUT_CHINA_DATA_PATH}")

if not df.empty and nlp:
    print("\n--- 步骤 1.1b: 开始并行预分词 ---")
    start_time_process = time.time()
    raw_token_lists = []

    # 禁用所有不需要的管道组件以获得最快速度
    disabled_pipes = [pipe for pipe in nlp.pipe_names if pipe not in ['tok2vec']]

    with nlp.select_pipes(disable=disabled_pipes):
        docs = nlp.pipe(df['CONTENT'], batch_size=1000, n_process=N_PROCESSES)
        for doc in tqdm(docs, total=len(df), desc="预分词中"):
            tokens = [token.text for token in doc if not token.is_punct and not token.is_space]
            raw_token_lists.append(tokens)

    print(f"\n预分词处理完成！耗时: {(time.time() - start_time_process)/60:.2f} 分钟。")

    print(f"正在保存预分词结果 ({len(raw_token_lists)}篇文章) 到 {TOKEN_LISTS_PATH}...")
    with open(TOKEN_LISTS_PATH, 'wb') as f:
        pickle.dump(raw_token_lists, f)
    print("保存成功！")
else:
    print("DataFrame为空或spaCy模型未加载，跳过处理。")

--- 步骤 1.1a: 数据加载 ---
数据加载完成！耗时: 0.04 秒。 数据形状: (1000, 2)

--- 步骤 1.1b: 开始并行预分词 ---


预分词中:   0%|          | 0/1000 [00:00<?, ?it/s]


预分词处理完成！耗时: 0.53 分钟。
正在保存预分词结果 (1000篇文章) 到 ../data_processed/intermediate_raw_token_lists.pkl...
保存成功！


### 步骤 1.2 - Gensim Phrases 发现

需求: 使用上一步的 raw_token_lists 训练 Phrases 模型。
实现: 从 .pkl 文件加载数据，然后进行训练和保存。

In [3]:
# --- 步骤 1.2: Gensim Phrases 发现 ---
if os.path.exists(TOKEN_LISTS_PATH):
    print(f"--- 步骤 1.2: 开始 Gensim Phrases 发现 ---")
    print(f"正在从 {TOKEN_LISTS_PATH} 加载预分词结果...")
    with open(TOKEN_LISTS_PATH, 'rb') as f:
        raw_token_lists_gensim = pickle.load(f)
    print("加载完成。")

    print("正在为Gensim Phrases准备小写化的token列表...")
    documents_for_gensim = [[token.lower() for token in doc] for doc in tqdm(raw_token_lists_gensim, desc="转小写")]
    del raw_token_lists_gensim

    print("\n开始训练Phrases模型...")
    start_time = time.time()
    min_count = 5 if TEST_MODE else 20
    print(f"使用 min_count = {min_count}")
    phrases_model = Phrases(documents_for_gensim, min_count=min_count, threshold=10.0, delimiter='_')
    train_time = time.time() - start_time
    print(f"Phrases模型训练完成！耗时: {train_time:.2f} 秒。")

    print("\n正在导出Gensim发现的候选短语...")
    gensim_candidates_raw = phrases_model.export_phrases()

    # 使用最简单、最兼容的二元组解包
    gensim_candidates = {
        phrase.replace('_', ' '): score
        for phrase, score in gensim_candidates_raw
    }

    print(f"Gensim Phrases发现了 {len(gensim_candidates)} 个候选短语。")

    print(f"\n正在将Gensim候选结果保存到 {CANDIDATES_GENSIM_PATH}...")
    with open(CANDIDATES_GENSIM_PATH, 'wb') as f:
        pickle.dump(gensim_candidates, f)
    print("保存成功！")
else:
    print(f"❌ 错误: Gensim的输入文件 {TOKEN_LISTS_PATH} 不存在。请先运行步骤 1.1。")

--- 步骤 1.2: 开始 Gensim Phrases 发现 ---
正在从 ../data_processed/intermediate_raw_token_lists.pkl 加载预分词结果...
加载完成。
正在为Gensim Phrases准备小写化的token列表...


转小写:   0%|          | 0/1000 [00:00<?, ?it/s]


开始训练Phrases模型...
使用 min_count = 5
Phrases模型训练完成！耗时: 0.67 秒。

正在导出Gensim发现的候选短语...
检测到Gensim返回字典格式，使用 .items() 进行解包。
Gensim Phrases发现了 3262 个候选短语。

正在将Gensim候选结果保存到 ../data_processed/candidates_gensim.pkl...
保存成功！


### 步骤 1.3 - 命名实体识别 (NER)

需求: 从原文中并行提取命名实体。
实现: 使用 nlp.pipe 并行处理，只启用 NER 相关组件。

In [4]:
# --- 步骤 1.3: 并行命名实体识别 (NER) ---

if 'df' in locals() and not df.empty and nlp:
    print("\n--- 步骤 1.3: 开始并行NER提取 ---")
    start_time_process = time.time()
    ner_candidates = Counter()

    print(f"将使用 {N_PROCESSES} 个进程。")
    TARGET_ENTITY_LABELS = {'ORG', 'PERSON', 'GPE', 'NORP', 'FAC', 'LOC', 'PRODUCT', 'EVENT'}

    # 专注于 NER, 启用其依赖的组件
    with nlp.select_pipes(enable=["tok2vec", "tagger", "ner"]):
        docs = nlp.pipe(df['CONTENT'], batch_size=500, n_process=N_PROCESSES)
        for doc in tqdm(docs, total=len(df), desc="NER提取中"):
            for ent in doc.ents:
                if ent.label_ in TARGET_ENTITY_LABELS and ' ' in ent.text:
                    ner_candidates[ent.text.strip().lower()] += 1

    print(f"\nNER提取完成！耗时: {(time.time() - start_time_process)/60:.2f} 分钟。")

    print(f"正在保存NER候选结果 ({len(ner_candidates)}个) 到 {CANDIDATES_NER_PATH}...")
    with open(CANDIDATES_NER_PATH, 'wb') as f:
        pickle.dump(ner_candidates, f)
    print("保存成功！")

    # 所有任务完成，清理DataFrame
    del ner_candidates
    del df
else:
    print("DataFrame (df) 未加载或为空，跳过NER提取。")

print("\n\n✅✅✅ 阶段一（精简版）全部完成！✅✅✅")
print("Gensim 和 NER 两种来源的候选短语都已生成并保存。")
print("下一步是运行后续的Notebook来整合和审核这些结果。")


--- 步骤 1.3: 开始并行NER提取 ---
将使用 5 个进程。


NER提取中:   0%|          | 0/1000 [00:00<?, ?it/s]


NER提取完成！耗时: 0.56 分钟。
正在保存NER候选结果 (8493个) 到 ../data_processed/candidates_ner.pkl...
保存成功！
